In [23]:
from sentence_transformers import SentenceTransformer, util

# Load the pre-trained BERT model
model = SentenceTransformer('thenlper/gte-base')


In [24]:
sentence1 = "I love you"
sentence2 = "You score a terrible 1!"

# Encode the sentences
embedding1 = model.encode(sentence1, convert_to_tensor=True)
embedding2 = model.encode(sentence2, convert_to_tensor=True)

# Compute the cosine similarity
cosine_score = util.pytorch_cos_sim(embedding1, embedding2)

print(f"Semantic score: {cosine_score.item()}")

Semantic score: 0.7613182067871094
